	ber-language

	<program>		::= DISAS <var-decls> <statements> .leo

	<var-decls>		::= <var-decl> |
					<var-decl> <var-decls>

	<var-decl>		::= <type> REGISTER ;

				
	<type> 			::= long | int | hex | bin | ;

	<statements>		::= <open_statement> |
					<open_statement> <statements>

	<statement>		::= <atrib> |
					<if-else>

	<atrib>			::= REGISTER = <expression> ;

	<expression>		::= REGISTER |
					number |
					addw <expression>, <expression> |
					subw <expression>, <expression> |
					mul <expression>, <expression> |
					div <expression>, <expression> |
					( <expression> )





	<open_statement>	::= <atrib> |
				se-o-ber ( <expression> <comp> <expression> ){ <open_statement> } |
				se-o-ber ( <expression> <comp> <expression> ){ <closed_statement> } senao-o-ber { <open_statement> }



	<closed_statement>	::= <atrib> |
				se-o-ber ( <expression> <comp> <expression> ){ <closed_statement> } senao-o-ber { <closed_statement> }



					== |  != |   <   |   >  |     <=    |    >=
	<comp>			::= leo | ber | lucca | enzo | lucca-leo | enzo-leo





In [403]:
!pip install rply --quiet


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


**Implementação do Analisador Léxico**

In [404]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('BINUMBER', r'0b[01]+')
lg.add('HEXNUMBER', r'0x[0-9A-Fa-f]+')
lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')
lg.add('OPEN_BRACKET', r'\{')
lg.add('CLOSE_BRACKET', r'\}')

lg.add('LAMBDA','lambda')

lg.add('INT', r'int')
lg.add('BIN', r'bin')
lg.add('HEX', r'hex')
lg.add('IF', r'se-o-ber')
lg.add('ELSE', r'senao-o-ber')
lg.add('WHILE', r'enquanto-o-ber')

lg.add('START', 'disas')
lg.add('END', '\.leo')
lg.add('COMP','leo')
lg.add('COMP','ber')
lg.add('COMP','enzo-leo')
lg.add('COMP','enzo')
lg.add('COMP','lucca-leo')
lg.add('COMP','lucca')
lg.add('REGISTER', r'[a-zA-Z][a-zA-Z0-9]*')

lg.add('EQUALS', r'=')
lg.add('SEMICOL', r';')
lg.add('DOT','\.')

lg.ignore('\s+')

lexer = lg.build()

In [405]:
program = """
disas
    bin REGISTER1;

    REGISTER1 = 0b101010;
.leo
"""



a = lexer.lex(program)

for i in a:
    print(i)

Token('START', 'disas')
Token('BIN', 'bin')
Token('REGISTER', 'REGISTER1')
Token('SEMICOL', ';')
Token('REGISTER', 'REGISTER1')
Token('EQUALS', '=')
Token('BINUMBER', '0b101010')
Token('SEMICOL', ';')
Token('END', '.leo')


In [406]:
program2 = """
disas
    int REGISTER1;
    bin REGISTER2;
    hex REGISTER3;

    se-o-ber (REGISTER1 leo 10) {
        REGISTER1 = 20;
    } senao-o-ber {
        REGISTER2 = 0b101010;
    }
    
    enquanto-o-ber (REGISTER2 lucca 0b111111) {
        REGISTER2 = REGISTER2 + 0b1;
    }

    REGISTER3 = 0x1af;
    
.leo
"""

a = lexer.lex(program2)

for i in a:
    print(i)

Token('START', 'disas')
Token('INT', 'int')
Token('REGISTER', 'REGISTER1')
Token('SEMICOL', ';')
Token('BIN', 'bin')
Token('REGISTER', 'REGISTER2')
Token('SEMICOL', ';')
Token('HEX', 'hex')
Token('REGISTER', 'REGISTER3')
Token('SEMICOL', ';')
Token('IF', 'se-o-ber')
Token('OPEN_PARENS', '(')
Token('REGISTER', 'REGISTER1')
Token('COMP', 'leo')
Token('NUMBER', '10')
Token('CLOSE_PARENS', ')')
Token('OPEN_BRACKET', '{')
Token('REGISTER', 'REGISTER1')
Token('EQUALS', '=')
Token('NUMBER', '20')
Token('SEMICOL', ';')
Token('CLOSE_BRACKET', '}')
Token('ELSE', 'senao-o-ber')
Token('OPEN_BRACKET', '{')
Token('REGISTER', 'REGISTER2')
Token('EQUALS', '=')
Token('BINUMBER', '0b101010')
Token('SEMICOL', ';')
Token('CLOSE_BRACKET', '}')
Token('WHILE', 'enquanto-o-ber')
Token('OPEN_PARENS', '(')
Token('REGISTER', 'REGISTER2')
Token('COMP', 'lucca')
Token('BINUMBER', '0b111111')
Token('CLOSE_PARENS', ')')
Token('OPEN_BRACKET', '{')
Token('REGISTER', 'REGISTER2')
Token('EQUALS', '=')
Token('REGISTER', 

In [407]:
program3 = """
disas

    int R;

    int REGISTER4 = lambda x. x+1;

    R = REGISTER4(3);
    
.leo
"""

In [408]:
from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self, decls,stmts):
        self.decls = decls
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_prog(self)

class VarDecls(BaseBox):
    def __init__(self, decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_vardecls(self)

class VarDecl(BaseBox):
    def __init__(self, id,tp,expr):
        self.id = id
        self.tp = tp
        self.expr = expr


    def accept(self, visitor):
        visitor.visit_vardecl(self)

class FuncDecl(BaseBox):
    def __init__(self, id, var,tp,expr):
        self.id = id
        self.var = var
        self.tp = tp
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_funcdecl(self)

class FuncCall(BaseBox):
    def __init__(self,id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_funccall(self)

class Statements(BaseBox):
    def __init__(self, stmt,stmts):
        self.stmt = stmt
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_statements(self)

class Statement(BaseBox):
    def __init__(self, cmd):
        self.cmd = cmd

    def accept(self, visitor):
        visitor.visit_statement(self)

class Atrib(BaseBox):
    def __init__(self, id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class IfElse(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1,ie2):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1
        self.ie2=ie2

    def accept(self, visitor):
        visitor.visit_ifelse(self)


class While(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1


    def accept(self, visitor):
        visitor.visit_while(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Register(Expr):
    def __init__(self, value):
        self.value = value

class BinNumber(Expr):
    def __init__(self, value):
        self.value = value

class HexNumber(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value


class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
  pass


class Sub(BinaryOp):
  pass


class Mul(BinaryOp):
  pass


class Div(BinaryOp):
  pass


**Implementando o Analisador Sintático**

In [409]:
from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['BINUMBER', 'HEXNUMBER', 'NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS', 'OPEN_BRACKET', 'CLOSE_BRACKET',
     'PLUS', 'MINUS', 'MUL', 'DIV',
     'INT', 'BIN', 'HEX', 'IF', 'ELSE', 'WHILE', 'START', 'END',
     'COMP', 'EQUALS', 'SEMICOL', 'REGISTER', 'DOT', 'LAMBDA'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])
    ]
)

@pg.production('prog : START decls statements END')
def prog(p):
    return Prog(p[1],p[2])

##################################################
# DECLARAÇÕES DE VARIÁVEIS
##################################################

@pg.production('decls : decl')
def vardecls(p):
    return VarDecls(p[0],None)

@pg.production('decls : decl decls')
def vardecls(p):
    return VarDecls(p[0],p[1])

@pg.production('decl : INT REGISTER SEMICOL')
def vardecl_int(p):
    return VarDecl(p[1].getstr(), "int", None)

@pg.production('decl : HEX REGISTER SEMICOL')
def vardecl_hex(p):
    return VarDecl(p[1].getstr(), "hex", None)

@pg.production('decl : BIN REGISTER SEMICOL')
def vardecl_bin(p):
    return VarDecl(p[1].getstr(), "bin", None)

@pg.production('decl : INT REGISTER EQUALS expression SEMICOL')
def vardecl_int_expr(p):
    return VarDecl(p[1].getstr(), "int", p[3])

@pg.production('decl : HEX REGISTER EQUALS expression SEMICOL')
def vardecl_hex_expr(p):
    return VarDecl(p[1].getstr(), "hex", p[3])

@pg.production('decl : BIN REGISTER EQUALS expression SEMICOL')
def vardecl_bin_expr(p):
    return VarDecl(p[1].getstr(), "bin", p[3])

##################################################
# FUNÇÕES
##################################################

@pg.production('decl : INT REGISTER EQUALS LAMBDA REGISTER DOT expression SEMICOL')
def FuncDecl_int_expr_func(p):
    return FuncDecl(p[1].getstr(), p[4].getstr(),"int", p[6])

##################################################
# COMANDOS - CASO ABERTO
##################################################

@pg.production('statements : statement')
def statement_statements(p):
    return Statements(p[0],None)

@pg.production('statements : statement statements')
def statement_statements(p):
    return Statements(p[0],p[1])

@pg.production('statement : REGISTER EQUALS expression SEMICOL')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('statement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS OPEN_BRACKET statement CLOSE_BRACKET')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[7],None)

@pg.production('statement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS OPEN_BRACKET statement CLOSE_BRACKET ELSE OPEN_BRACKET statement CLOSE_BRACKET')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[7],p[11])

@pg.production('statement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS OPEN_BRACKET statement CLOSE_BRACKET')
def statement_while(p):
    return While (p[2],p[3],p[4],p[7])

@pg.production('expression : REGISTER')
def expression_id(p):
    return Register(p[0].getstr())

@pg.production('expression : BINUMBER')
def expression_binumber(p):
    return BinNumber(p[0].getstr())

@pg.production('expression : HEXNUMBER')
def expression_hexnumber(p):
    return HexNumber(p[0].getstr())

@pg.production('expression : NUMBER')
def expression_number(p):
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : REGISTER OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return FuncCall(p[0].getstr(),p[2])

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

**Visitor da Árvore Sintática**

In [410]:

ST={}

FT={}

class Visitor(object):
  pass

class SymbolTable(Visitor):
    def visit_prog(self, prog):
        prog.decls.accept(self)

    def visit_vardecls(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)

    def visit_vardecl(self, d):
        ST[d.id]=d.tp

    def visit_funcdecl(self,d):
        FT[d.id]=(d.var,d.expr)

In [411]:
symbol = SymbolTable()
tree = parser.parse(lexer.lex(program2))
tree.accept(symbol)
print(ST)

{'REGISTER1': 'int', 'REGISTER2': 'bin', 'REGISTER3': 'hex'}


In [412]:
symbol = SymbolTable()
tree = parser.parse(lexer.lex(program3))
tree.accept(symbol)
print(FT)

{'REGISTER4': ('x', <__main__.Add object at 0x00000268DD7BE3B0>)}


In [413]:
#VISITOR PARA DECORAR TIPOS

class Decorator(Visitor):

    def visit_prog(self, i):
        i.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_atrib(self, i):
        if i.id in ST:
          i.decor_type=ST[i.id]
        else:
          raise AssertionError('id not declared')
        i.expr.accept(self)

    def visit_funccall(self, i):
      if i.id in FT.keys():
        i.decor_type = 'int'
        i.expr.accept(self)
      else:
          raise AssertionError('function not declared')

    def visit_ifelse(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)
        if i.ie2!=None:
          i.ie2.accept(self)

    def visit_while(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)


    def visit_register(self, i):
        if i.value in ST:
          i.decor_type=ST[i.value]
        else:
          raise AssertionError('register not declared')


    def visit_number(self, i):
        i.decor_type="int"
    
    def visit_binnumber(self, i):
        i.decor_type="bin"

    def visit_hexnumber(self, i):
        i.decor_type="hex"


    def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"


    def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"

    def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"

    def visit_div(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"



In [414]:
# VISITOR - TYPE VERIFIER

class TypeVerifier(Visitor):

    def visit_prog(self, i):
        i.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_funccall(self, d):
       if d.expr.decor_type != 'int':
          raise AssertionError('type error')

    def visit_atrib(self, i):
        if i.decor_type!=i.expr.decor_type:
          raise AssertionError('type error')

    def visit_ifelse(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')

    def visit_while(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')

In [415]:
arvore=parser.parse(lexer.lex(program2))
arvore.accept(SymbolTable())
arvore.accept(Decorator())
arvore.accept(TypeVerifier())

In [416]:
#VISITOR PARA GERAR CÓDIGO INTERMEDIÁRIO (1.0 PONTO)

class IntermediateCode(Visitor):

  def __init__(self):
        self.level=0  #nível do if-then-else
        self.loop=0  #número do loop
        self.comp={"leo":"8","ber":"9","lucca":"10", "lucca-leo":"11", "enzo":"12", "enzo-leo":"13" } #códigos dos comparadores


  def visit_prog(self, i):
        i.decls.accept(self)
        i.stmts.accept(self)

  def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)


  def visit_statement(self, d):
        d.cmd.accept(self)

  def visit_vardecls(self, i):
        i.decl.accept(self)
        if i.decls != None:
          i.decls.accept(self)

  def visit_vardecl(self, i):
        if i.expr != None:
          i.expr.accept(self)
          print("sto 0",i.id)

  def visit_funcdecl(self, i):
      pass

  def visit_funccall(self, i):
        i.expr.accept(self)

        var,expr = FT[i.id]
        
        expr.accept(self)


  def visit_atrib(self, i):
        i.expr.accept(self)
        print("sto 0",i.id)

  def visit_ifelse(self, i):
        i.expr2.accept(self) #gera código para expressão 2
        i.expr1.accept(self) #gera código para expressão 1
        print("opr 0 "+self.comp[i.comp.getstr()])  #obtém o código do operador de comparação
        self.level=self.level+1
        print("jpc 0 true"+str(self.level)) # se o resultado da comparação for verdadeiro, pula para o true correspondente
        print("jmp 0 false"+str(self.level))# se o resultado da comparação for falso, pula para o true correspondente
        print("true"+str(self.level)+":") #gera o rótulo da seção true
        i.ie1.accept(self) # gera o código para true
        print("jmp final"+str(self.level)) # pula para o final do bloco if-true-else
        print("false"+str(self.level)+":") #gera o rótulo para a seção false
        if i.ie2!=None:  # se houver else, gera o código
          i.ie2.accept(self)
        print("final"+str(self.level)+":") #gera o rótulo para o final do bloco
        self.level=self.level-1

  def visit_while(self, i):
        self.loop=self.loop+1
        print("loop"+str(self.loop)+":") #gera o rótulo da seção true
        i.expr2.accept(self) #gera código para expressão 2
        i.expr1.accept(self) #gera código para expressão 1
        print("opr 0 "+self.comp[i.comp.getstr()])  #obtém o código do operador de comparação
        self.level=self.level+1
        print("jpc 0 true"+str(self.level)) # se o resultado da comparação for verdadeiro, pula para o true correspondente
        print("jmp 0 final"+str(self.level))# se o resultado da comparação for falso, pula para o true correspondente
        print("true"+str(self.level)+":") #gera o rótulo da seção true
        i.ie1.accept(self) # gera o código para true
        print("jmp loop"+str(self.loop)) # pula para o início do loop
        print("final"+str(self.level)+":") #gera o rótulo para o final do bloco do loop
        self.level=self.level-1


  def visit_number(self, i):
    print("lit 0 ",i.value)

  def visit_binnumber(self, i):
    print("lit 0 ",i.value)

  def visit_hexnumber(self, i):
    print("lit 0 ",i.value)

  def visit_register(self, i):
    print("lod 0 ",i.value)

  def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        print("opr 0 2")


  def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)
        print("opr 0 3")


  def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        print("opr 0 4")


  def visit_div(self, a):
        a.left.accept(self)
        a.right.accept(self)
        print("opr 0 5")


In [417]:
arvore=parser.parse(lexer.lex(program2))
arvore.accept(SymbolTable())
arvore.accept(Decorator())
arvore.accept(TypeVerifier())
arvore.accept(IntermediateCode())

lit 0  10
lod 0  REGISTER1
opr 0 8
jpc 0 true1
jmp 0 false1
true1:
lit 0  20
sto 0 REGISTER1
jmp final1
false1:
lit 0  0b101010
sto 0 REGISTER2
final1:
loop1:
lit 0  0b111111
lod 0  REGISTER2
opr 0 10
jpc 0 true1
jmp 0 final1
true1:
lod 0  REGISTER2
lit 0  0b1
opr 0 2
sto 0 REGISTER2
jmp loop1
final1:
lit 0  0x1af
sto 0 REGISTER3


In [418]:
arvore=parser.parse(lexer.lex(program3))
arvore.accept(SymbolTable())
arvore.accept(Decorator())
arvore.accept(TypeVerifier())
arvore.accept(IntermediateCode())

lit 0  3
lod 0  x
lit 0  1
opr 0 2
sto 0 R
